In [1]:
import neptune
import os
import re
import pandas as pd

In [2]:
PROJECT_NAME = "kofmanya/HSE-MDS-Kofman-Anna-Diploma"
API_TOKEN = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4MmUyOTZjYy1mNzFjLTQ4YzUtYjk4Yi1hZmIxMTk5OWYwMDgifQ=="

In [4]:
PROD_DESTINATION_DIR = "prod_artifacts"

In [6]:
def promote_best_model_to_prod():
    project = neptune.init_project(
        project=PROJECT_NAME,
        api_token=API_TOKEN
    )
    runs_df = project.fetch_runs_table().to_pandas()
    eval_candidates = []

    def parse_model_name_from_test_col(column_name):
        match = re.match(r"^test/(.+)_accuracy$", column_name)
        return match.group(1) if match else None

    for idx, row in runs_df.iterrows():
        run_id = row["sys/id"]
        for col in runs_df.columns:
            val = row[col]
            if (
                isinstance(val, (int, float))  
                and not pd.isna(val)          
                and col.startswith("test/")
                and col.endswith("_accuracy")
            ):
                model_name = parse_model_name_from_test_col(col)
                if model_name:
                    accuracy_val = float(val)
                    eval_candidates.append((run_id, model_name, accuracy_val))

    if not eval_candidates:
        print("No runs found that contain a 'test/<model>_accuracy' metric. Exiting.")
        return

    best_eval_run_id, best_model_name, best_accuracy = max(eval_candidates, key=lambda x: x[2])
    print(f"Best accuracy overall:")
    print(f" - Run: {best_eval_run_id}")
    print(f" - Model: {best_model_name}")
    print(f" - Accuracy: {best_accuracy:.4f}")

    model_folder = os.path.join(PROD_DESTINATION_DIR, best_model_name)
    os.makedirs(model_folder, exist_ok=True)
    
    def find_and_download_artifact(artifact_path, local_filepath):
        for i2, row2 in runs_df.iterrows():
            r2_id = row2["sys/id"]
            with neptune.init_run(
                project=PROJECT_NAME,
                with_id=r2_id,
                api_token=API_TOKEN,
                mode="read-only"
            ) as run_check:
                if run_check.exists(artifact_path):
                    print(f"  Found '{artifact_path}' in run {r2_id}, downloading -> {local_filepath}")
                    run_check[artifact_path].download(local_filepath)
                    return True
        return False

    weights_key = f"model/{best_model_name}_weights"
    embeddings_key = f"embeddings/{best_model_name}/path"
    transformations_key = f"transformations/{best_model_name}/path"

    weights_local = os.path.join(model_folder, f"{best_model_name}.pth")
    embeddings_local = os.path.join(model_folder, f"{best_model_name}_embeddings.npz")
    transformations_local = os.path.join(model_folder, f"{best_model_name}_transformations.json")

    print(f"\nLooking for weights artifact '{weights_key}'...")
    got_weights = find_and_download_artifact(weights_key, weights_local)
    if not got_weights:
        print(f"WARNING: Could not find any run with artifact '{weights_key}'.")

    print(f"\nLooking for embeddings artifact '{embeddings_key}'...")
    got_embeddings = find_and_download_artifact(embeddings_key, embeddings_local)
    if not got_embeddings:
        print(f"WARNING: Could not find any run with artifact '{embeddings_key}'.")

    print(f"\nLooking for transformations artifact '{transformations_key}'...")
    got_transformations = find_and_download_artifact(transformations_key, transformations_local)
    if not got_transformations:
        print(f"WARNING: Could not find any run with artifact '{transformations_key}'.")

    prod_run = neptune.init_run(
        project=PROJECT_NAME,
        api_token=API_TOKEN,
        name="Production Model Artifacts (Best Model)"
    )

    if os.path.exists(weights_local):
        print(f"Uploading {weights_local} to Neptune -> 'prod/{best_model_name}'")
        prod_run[f"prod/{best_model_name}"].upload(weights_local)

    if os.path.exists(embeddings_local):
        print(f"Uploading {embeddings_local} to Neptune -> 'prod/{best_model_name}_embeddings'")
        prod_run[f"prod/{best_model_name}_embeddings"].upload(embeddings_local)

    if os.path.exists(transformations_local):
        print(f"Uploading {transformations_local} to Neptune -> 'prod/{best_model_name}_transformations'")
        prod_run[f"prod/{best_model_name}_transformations"].upload(transformations_local)

    prod_run.stop()

    print("\n=== Done! ===")
    print(f"Best model: '{best_model_name}' (accuracy={best_accuracy:.4f})")
    print(f"Local artifacts saved under: {model_folder}")
    print("Production artifacts uploaded to Neptune in a single run under 'prod/<model>...'.")

promote_best_model_to_prod()

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/kofmanya/HSE-MDS-Kofman-Anna-Diploma/
Best accuracy overall:
 - Run: HSEM-21
 - Model: vit
 - Accuracy: 0.8765

Looking for weights artifact 'model/vit_weights'...
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/kofmanya/HSE-MDS-Kofman-Anna-Diploma/e/HSEM-26
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/kofmanya/HSE-MDS-Kofman-Anna-Diploma/e/HSEM-26/metadata
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/kofmanya/HSE-MDS-Kofman-Anna-Diploma/e/HSEM-25
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/kofmanya/HSE-MDS-Kofman-Anna-Diploma/e/HSEM-25/metadata
[neptune] [info  

Fetching file...: 0 [00:00, ?/s]

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/kofmanya/HSE-MDS-Kofman-Anna-Diploma/e/HSEM-20/metadata

Looking for embeddings artifact 'embeddings/vit/path'...
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/kofmanya/HSE-MDS-Kofman-Anna-Diploma/e/HSEM-26
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/kofmanya/HSE-MDS-Kofman-Anna-Diploma/e/HSEM-26/metadata
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/kofmanya/HSE-MDS-Kofman-Anna-Diploma/e/HSEM-25
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/kofmanya/HSE-MDS-Ko

Fetching file...: 0 [00:00, ?/s]

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/kofmanya/HSE-MDS-Kofman-Anna-Diploma/e/HSEM-22/metadata

Looking for transformations artifact 'transformations/vit/path'...
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/kofmanya/HSE-MDS-Kofman-Anna-Diploma/e/HSEM-26
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/kofmanya/HSE-MDS-Kofman-Anna-Diploma/e/HSEM-26/metadata
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/kofmanya/HSE-MDS-Kofman-Anna-Diploma/e/HSEM-25
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/kofmanya/